# Fazendo uma nova DS
Pelo add data, consigo adicionar a competição rsna-intracranial-hemorrhage-detection ao notebook e acessar os dados sem precisar baixar!

## Imports e Instalações

In [1]:
!mkdir ~/.kaggle
!cp ../input/kagglejson/kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
import pandas as pd
import kaggle
import shutil

In [3]:
input_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection"
train_images_dir = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/"
#!ls {input_path}

### Criando os diretorios

In [4]:
!mkdir small_rsna_ds
!mkdir small_rsna_ds/yes
!mkdir small_rsna_ds/no

## Selecionando os arquivos
*   https://www.kaggle.com/code/akensert/rsna-inceptionv3-keras-tf1-14-0?scriptVersionId=39570832 secção 4



### Definições

In [5]:
def read_trainset(filename=input_path+"/stage_2_train.csv"):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        56346, 56347, 56348, 56349,
        56350, 56351, 1171830, 1171831,
        1171832, 1171833, 1171834, 1171835,
        3705312, 3705313, 3705314, 3705315,
        3705316, 3705317, 3842478, 3842479,
        3842480, 3842481, 3842482, 3842483
    ]
    
    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

df = read_trainset()

In [6]:
df

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_000012eaf     0        0                0                0            0   
ID_000039fa0     0        0                0                0            0   
ID_00005679d     0        0                0                0            0   
ID_00008ce3c     0        0                0                0            0   
ID_0000950d7     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_ffff82e46     0        0                0                0            0   
ID_ffff922b9     1        0                0                1            0   
ID_ffffb670a     1        0                0                0            1   
ID_ffffcbff8     0        0                0                0            0   
ID_fffff9393     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_000012eaf        0  
ID_000039fa0        0  
ID_00005679d        0  
ID_00008ce3c        0  
ID_0000950d7        0  
...               ...  
ID_ffff82e46        0  
ID_ffff922b9        0  
ID_ffffb670a        0  
ID_ffffcbff8        0  
ID_fffff9393        0  

[752803 rows x 6 columns]

### Selecionando aleatoriamente

In [7]:
df_shuffled = df.sample(frac=1)
df_shuffled

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_097eef903     0        0                0                0            0   
ID_b0617a470     0        0                0                0            0   
ID_9822390ab     0        0                0                0            0   
ID_afc4b0737     0        0                0                0            0   
ID_d9e71a253     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_8cfa55ff1     0        0                0                0            0   
ID_be74da96b     0        0                0                0            0   
ID_13e2be97a     0        0                0                0            0   
ID_00cac32ca     0        0                0                0            0   
ID_08cb43958     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_097eef903        0  
ID_b0617a470        0  
ID_9822390ab        0  
ID_afc4b0737        0  
ID_d9e71a253        0  
...               ...  
ID_8cfa55ff1        0  
ID_be74da96b        0  
ID_13e2be97a        0  
ID_00cac32ca        0  
ID_08cb43958        0  

[752803 rows x 6 columns]

In [8]:
df_shorten = df_shuffled[:35000]
df_shorten

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
Image                                                                        
ID_097eef903     0        0                0                0            0   
ID_b0617a470     0        0                0                0            0   
ID_9822390ab     0        0                0                0            0   
ID_afc4b0737     0        0                0                0            0   
ID_d9e71a253     0        0                0                0            0   
...            ...      ...              ...              ...          ...   
ID_c9f9d5286     0        0                0                0            0   
ID_ecec001ca     0        0                0                0            0   
ID_49bdef137     0        0                0                0            0   
ID_9dd087d1f     0        0                0                0            0   
ID_89e45b4b3     0        0                0                0            0   

                       
Diagnosis    subdural  
Image                  
ID_097eef903        0  
ID_b0617a470        0  
ID_9822390ab        0  
ID_afc4b0737        0  
ID_d9e71a253        0  
...               ...  
ID_c9f9d5286        0  
ID_ecec001ca        0  
ID_49bdef137        0  
ID_9dd087d1f        0  
ID_89e45b4b3        0  

[35000 rows x 6 columns]

## Criando o novo Dataset

In [9]:
n = 0
while n < 34999:
    id = df_shorten.index[n]
    hemorrhage = df.loc[[id]].iat[0,0]
    
    src = train_images_dir+id+'.dcm'
    
    if(hemorrhage == 1):
        dst = './small_rsna_ds/yes/'
    else:
        dst = './small_rsna_ds/no/'
    
    shutil.copy(src, dst)
    n = n+1

In [10]:
!zip -rm ds_35000.zip ./small_rsna_ds

  adding: small_rsna_ds/ (stored 0%)
  adding: small_rsna_ds/no/ (stored 0%)
  adding: small_rsna_ds/no/ID_e0cf95162.dcm (deflated 49%)
  adding: small_rsna_ds/no/ID_6c2f5ca89.dcm (deflated 54%)
  adding: small_rsna_ds/no/ID_57d71ec1f.dcm (deflated 55%)
  adding: small_rsna_ds/no/ID_024db6aad.dcm (deflated 61%)
  adding: small_rsna_ds/no/ID_afbf32a0e.dcm (deflated 66%)
  adding: small_rsna_ds/no/ID_44e37b7b0.dcm (deflated 56%)
  adding: small_rsna_ds/no/ID_fa2769aec.dcm (deflated 63%)
  adding: small_rsna_ds/no/ID_2d07aac23.dcm (deflated 54%)
  adding: small_rsna_ds/no/ID_b7b4f393c.dcm (deflated 58%)
  adding: small_rsna_ds/no/ID_e24083229.dcm (deflated 56%)
  adding: small_rsna_ds/no/ID_afa325baf.dcm (deflated 65%)
  adding: small_rsna_ds/no/ID_e2a33c4fe.dcm (deflated 67%)
  adding: small_rsna_ds/no/ID_76bea99c7.dcm (deflated 54%)
  adding: small_rsna_ds/no/ID_8fe334555.dcm (deflated 56%)
  adding: small_rsna_ds/no/ID_0542710dc.dcm (deflated 46%)
  adding: small_rsna_ds/no/ID_fd23961d